In [1]:
from urllib.request import urlopen 
from bs4 import BeautifulSoup,Comment
import pandas as pd
import numpy as np
import json
import requests
%run rookie_scraper.ipynb
import matplotlib.pyplot as plt

Below is a parser that helps create an easy html lookup from basketball reference, the parser will parse the first letter of the last name (lowercase), and create the id that will help with the html lookup

In [2]:
def roy_parser(name):
    fn_ln= name.split(" ")
    first_letter_of_last_name= fn_ln[1][:1].lower()
    first_two_letters_of_first_name= fn_ln[0][:2].lower()
    last_name_lower= fn_ln[1][:5].lower()
    html_look_up= last_name_lower + first_two_letters_of_first_name + "01"
    return first_letter_of_last_name,html_look_up

In [58]:
def get_stats(hyperlink):
    url= hyperlink
    html = urlopen(url)
    soup = BeautifulSoup(html,'lxml')
    
    headers = [th.getText() for th in soup.findAll('tr',limit=2)[0].findAll('th')]
    headers = headers[1:]
    
    rows = soup.findAll('tr')[1:]
    player_stats = [[td.getText() for td in rows[i].findAll('td')] for i in range(len(rows))]
    stats = pd.DataFrame(player_stats, columns = headers)
    if stats.shape[0]>5:
        stats= stats.head()
    num_stats= list(stats.columns.values)[4:]
    #converting numerical stat values to floats
    for stat in num_stats:
        stats[stat]= pd.to_numeric(stats[stat], errors='coerce')
    return stats.drop_duplicates().dropna(how='all')

In [59]:
roys={}
for i in range(len(links_of_roy_votes)):
    roys[names_of_roy_votes[i]]= get_stats(links_of_roy_votes[i])

In [78]:
def get_advanced_stats(hyperlink):
    res = requests.get(hyperlink,headers={"User-Agent":"Mozilla/5.0"})
    soup = BeautifulSoup(res.text, 'lxml')
    keep=[]
    for comment in soup.find_all(string=lambda text:isinstance(text,Comment)):
        data = BeautifulSoup(comment,"lxml")
        for items in data.select("table.row_summable tr"):
            tds = [item.get_text(strip=True) for item in items.select("th,td")]
            keep.append(tds)
    adv= ['Season', 'Age', 'Tm', 'Lg', 'Pos', 'G', 'MP', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', '', 'OWS', 'DWS', 'WS', 'WS/48', '', 'OBPM', 'DBPM', 'BPM', 'VORP']
    adv_df= pd.DataFrame(columns=adv,index=['S1','S2','S3','S4','S5'])
    for i in range(len(keep)):
        if (np.array_equal(keep[i],adv)):
            adv_df.loc['S1']= keep[i+1]
            if(keep[i+2][0]!='Career'):
                adv_df.loc['S2']= keep[i+2]
                if(keep[i+3][0]!='Career'):
                    adv_df.loc['S3']= keep[i+3]
                    if(keep[i+4][0]!='Career'):
                        adv_df.loc['S4']= keep[i+4]
                        if(keep[i+5][0]!='Career'):
                            adv_df.loc['S5']= keep[i+5]
            if True:
                break
     
    adv_df= adv_df.dropna()
    return adv_df

In [79]:
get_advanced_stats('https://www.basketball-reference.com/players/j/jamesle01.html')

,Season,Age,Tm,Lg,Pos,G,MP,PER,TS%,3PAr,...,,OWS,DWS,WS,WS/48,,OBPM,DBPM,BPM,VORP
S1,2003-04,19,CLE,NBA,SG,79,3122,18.3,.488,.145,...,,2.4,2.6,5.1,.078,,2.2,-0.2,1.9,3.1
S2,2004-05,20,CLE,NBA,SF,80,3388,25.7,.554,.183,...,,9.7,4.6,14.3,.203,,6.9,1.5,8.3,8.8
S3,2005-06,21,CLE,NBA,SF,79,3361,28.1,.568,.208,...,,12.0,4.3,16.3,.232,,7.9,1.4,9.3,9.5
S4,2006-07,22,CLE,NBA,SF,78,3190,24.5,.552,.191,...,,8.0,5.7,13.7,.206,,5.4,2.0,7.4,7.6
S5,2007-08,23,CLE,NBA,SF,75,3027,29.1,.568,.219,...,,10.7,4.6,15.2,.242,,9.0,2.3,11.2,10.1


In [39]:
get_advanced_stats(links_of_roy_votes[102])

,Season,Age,Tm,Lg,Pos,G,MP,PER,TS%,3PAr,...,,OWS,DWS,WS,WS/48,,OBPM,DBPM,BPM,VORP
S1,2006-07,29,TOR,NBA,PF,67,1909,12.1,.503,.365,...,,0.6,2.4,3.0,.074,,-0.8,1.7,0.8,1.4
S2,2007-08,30,TOR,NBA,PF,7,74,10.4,.440,.640,...,,0.0,0.1,0.1,.047,,-1.9,-0.8,-2.7,0.0


In [40]:
roys_adv={}
for i in range(len(links_of_roy_votes)):
    roys_adv[names_of_roy_votes[i]]= get_advanced_stats(links_of_roy_votes[i])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187


In [80]:
for name in names_of_roy_votes:
    roys_adv[name]['WS']= pd.to_numeric(roys_adv[name]['WS'], errors='coerce')
    roys_adv[name]['PER']= pd.to_numeric(roys_adv[name]['PER'], errors='coerce')

In [81]:
for name in names_of_roy_votes:
            roys[name]= roys[name][roys[name]['Age']!='']